# Setup

In [4]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/train_cleaned.csv")

# Actual Model

## Creating the encoder

In [15]:
NUM_ROWS = 10000
BATCH_SIZE = 64

MAX_TOKENS = 5000
MAX_LENGTH = 200
encoder = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS, output_sequence_length=MAX_LENGTH)
encoder.adapt(data.head(NUM_ROWS)["comment_text"].tolist())

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

len(vocab)

5000

## Creating the model

In [16]:
# Sets random seed so results are identical every time
SEED = 1
tf.random.set_seed(SEED)
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

# TOXIC_CATEGORIES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
TOXIC_CATEGORIES = ["toxic"]

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(TOXIC_CATEGORIES), activation="sigmoid")
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])


### Training the model

In [17]:
data.shape

(149348, 9)

In [18]:
# Classification for all types of toxicity
multiDf = data.head(NUM_ROWS)[["comment_text"] + TOXIC_CATEGORIES]

x = multiDf["comment_text"]
y = multiDf[TOXIC_CATEGORIES]

splitter = StratifiedShuffleSplit(random_state=1, test_size=0.2)

min_train_size = 1000
for train, test in splitter.split(x, y[TOXIC_CATEGORIES[0]]):
    if len(train) < min_train_size:
        continue
    training_data = x.iloc[train]
    training_target = y.iloc[train]
    validation_data = x.iloc[test]
    validation_target = y.iloc[test]
    break  # Stop after finding the first valid split


In [19]:
# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2)

history = model.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback], batch_size=32)

Epoch 1/10
250/250 [==============================] - 66s 184ms/step - loss: 0.4005 - accuracy: 0.8986 - val_loss: 0.2852 - val_accuracy: 0.9090
Epoch 2/10
250/250 [==============================] - 22s 89ms/step - loss: 0.2343 - accuracy: 0.9212 - val_loss: 0.1838 - val_accuracy: 0.9400
Epoch 3/10
250/250 [==============================] - 19s 76ms/step - loss: 0.1284 - accuracy: 0.9599 - val_loss: 0.1698 - val_accuracy: 0.9425
Epoch 4/10
250/250 [==============================] - 15s 61ms/step - loss: 0.0715 - accuracy: 0.9778 - val_loss: 0.1915 - val_accuracy: 0.9390
Epoch 5/10
250/250 [==============================] - 13s 54ms/step - loss: 0.0428 - accuracy: 0.9884 - val_loss: 0.2277 - val_accuracy: 0.9455
Epoch 6/10
250/250 [==============================] - 13s 52ms/step - loss: 0.0234 - accuracy: 0.9942 - val_loss: 0.2796 - val_accuracy: 0.9435
Epoch 7/10
250/250 [==============================] - 13s 53ms/step - loss: 0.0137 - accuracy: 0.9967 - val_loss: 0.3103 - val_accuracy

In [26]:
model_1 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01))),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, kernel_regularizer=tf.keras.regularizers.l2(0.01))),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(TOXIC_CATEGORIES), activation="sigmoid")
])


model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4)

history = model_1.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback], batch_size=32)

Epoch 1/10
250/250 [==============================] - 50s 136ms/step - loss: 11.2951 - accuracy: 0.8957 - val_loss: 7.1169 - val_accuracy: 0.9060
Epoch 2/10
250/250 [==============================] - 17s 69ms/step - loss: 4.8619 - accuracy: 0.9056 - val_loss: 3.1221 - val_accuracy: 0.9060
Epoch 3/10
250/250 [==============================] - 16s 65ms/step - loss: 2.2071 - accuracy: 0.9057 - val_loss: 1.4886 - val_accuracy: 0.9060
Epoch 4/10
250/250 [==============================] - 13s 52ms/step - loss: 1.1382 - accuracy: 0.9060 - val_loss: 0.8370 - val_accuracy: 0.9065
Epoch 5/10
250/250 [==============================] - 13s 50ms/step - loss: 0.6592 - accuracy: 0.9174 - val_loss: 0.5114 - val_accuracy: 0.9245
Epoch 6/10
250/250 [==============================] - 13s 50ms/step - loss: 0.4117 - accuracy: 0.9411 - val_loss: 0.3990 - val_accuracy: 0.9115
Epoch 7/10
250/250 [==============================] - 12s 49ms/step - loss: 0.2961 - accuracy: 0.9554 - val_loss: 0.3418 - val_accurac

In [28]:
# JUST adding 2 more dense layers to the initial model
model_2 =tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(TOXIC_CATEGORIES), activation="sigmoid")
])

model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4)

history = model_2.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback], batch_size=32)

Epoch 1/10
250/250 [==============================] - 85s 275ms/step - loss: 0.3939 - accuracy: 0.9038 - val_loss: 0.2848 - val_accuracy: 0.9060
Epoch 2/10
250/250 [==============================] - 18s 72ms/step - loss: 0.2263 - accuracy: 0.9057 - val_loss: 0.1917 - val_accuracy: 0.9060
Epoch 3/10
250/250 [==============================] - 15s 60ms/step - loss: 0.1398 - accuracy: 0.9171 - val_loss: 0.1894 - val_accuracy: 0.9395
Epoch 4/10
250/250 [==============================] - 13s 52ms/step - loss: 0.0872 - accuracy: 0.9710 - val_loss: 0.2423 - val_accuracy: 0.9415
Epoch 5/10
250/250 [==============================] - 12s 48ms/step - loss: 0.0478 - accuracy: 0.9868 - val_loss: 0.2964 - val_accuracy: 0.9450
Epoch 6/10
250/250 [==============================] - 11s 45ms/step - loss: 0.0294 - accuracy: 0.9910 - val_loss: 0.3669 - val_accuracy: 0.9420
Epoch 7/10
250/250 [==============================] - 12s 48ms/step - loss: 0.0157 - accuracy: 0.9964 - val_loss: 0.4041 - val_accuracy

In [29]:
#Increase the number of LSTM cells in the bidirectional LSTM layers. This can help t
# he model learn more complex temporal dependencies

# Different values for dropout
model_3 =tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(len(TOXIC_CATEGORIES), activation="sigmoid")
])

model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4)

history = model_3.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback], batch_size=32)

Epoch 1/10
250/250 [==============================] - 49s 133ms/step - loss: 0.3641 - accuracy: 0.9024 - val_loss: 0.2574 - val_accuracy: 0.9090
Epoch 2/10
250/250 [==============================] - 18s 73ms/step - loss: 0.1699 - accuracy: 0.9373 - val_loss: 0.1700 - val_accuracy: 0.9385
Epoch 3/10
250/250 [==============================] - 15s 61ms/step - loss: 0.0834 - accuracy: 0.9710 - val_loss: 0.1918 - val_accuracy: 0.9315
Epoch 4/10
250/250 [==============================] - 14s 56ms/step - loss: 0.0416 - accuracy: 0.9864 - val_loss: 0.2466 - val_accuracy: 0.9465
Epoch 5/10
250/250 [==============================] - 12s 50ms/step - loss: 0.0213 - accuracy: 0.9939 - val_loss: 0.2905 - val_accuracy: 0.9480
Epoch 6/10
250/250 [==============================] - 12s 47ms/step - loss: 0.0127 - accuracy: 0.9962 - val_loss: 0.3313 - val_accuracy: 0.9435
Epoch 7/10
250/250 [==============================] - 13s 52ms/step - loss: 0.0081 - accuracy: 0.9981 - val_loss: 0.4135 - val_accuracy

### Getting test data

In [ ]:
# Classification for all toxicity values
test_data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test.csv")
test_labels = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test_labels.csv")

test_labels = test_labels
merged_df = test_labels.merge(test_data, left_on="id", right_on="id")
merged_df = merged_df.loc[(merged_df["toxic"] != -1) & (merged_df["severe_toxic"] != -1) & (merged_df["obscene"] != -1) & (merged_df["threat"] != -1) & (merged_df["insult"] != -1) & (merged_df["identity_hate"] != -1)]

In [ ]:
# Tests all rows with a value of 0 or 1 for all toxicity values

test_df = merged_df["comment_text"]
test_target = merged_df[TOXIC_CATEGORIES]
model.evaluate(test_df, test_target)

In [ ]:
# Test where any toxicity value is 1

test_df = merged_df["comment_text"]
query = " | ".join([f"({label} == 1)" for label in TOXIC_CATEGORIES])
filtered_df = merged_df.query(query)

filtered_test_dataset = filtered_df["comment_text"]
filtered_df_target = filtered_df[TOXIC_CATEGORIES]
model.evaluate(filtered_test_dataset, filtered_df_target)

In [ ]:
# Test where all toxicity values are 0

test_df = merged_df["comment_text"]
query = " | ".join([f"({label} == 0)" for label in TOXIC_CATEGORIES])
filtered_df = merged_df.query(query)

filtered_test_dataset = filtered_df["comment_text"]
filtered_df_target = filtered_df[TOXIC_CATEGORIES]
model.evaluate(filtered_test_dataset, filtered_df_target)

In [ ]:
submission_set = pd.read_csv('/kaggle/input/toxic-message-classifier-dataset/test.csv')
submission_set.head()

x_test = submission_set['comment_text'].values
y_testing = model.predict(x_test, verbose=1, batch_size=BATCH_SIZE)

y_testing

In [ ]:
submission_df = pd.DataFrame(columns = ['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])

submission_df['id'] = submission_set['id']
submission_df['toxic'] = [0 if x[0] < 0.5 else 1 for x in y_testing]
submission_df['severe_toxic'] = [0 if x[1] < 0.5 else 1 for x in y_testing]
submission_df['obscene'] = [0 if x[2] < 0.5 else 1 for x in y_testing]
submission_df['threat'] = [0 if x[3] < 0.5 else 1 for x in y_testing]
submission_df['insult'] = [0 if x[4] < 0.5 else 1 for x in y_testing]
submission_df['identity_hate'] = [0 if x[5] < 0.5 else 1 for x in y_testing]

submission_df.head()

submission_df.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
model.summary()